In [1]:
!pip install numpy==1.22.1

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import wandb

In [2]:
wandb.login()   # Log into wandb to track computing.

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: michael-bemus (michael-bemus01). Use `wandb login --relogin` to force relogin


True

In [3]:
import numpy as np
import pandas as pd
import re

from IPython.display import clear_output

In [4]:
from sacrebleu.metrics import BLEU
bleu_scorer = BLEU(effective_order=True)

from rouge import Rouge
rouge_scorer = Rouge()

In [5]:
df = pd.read_csv("/home/x-mbemus/Desktop/text_train_data.csv")   # Read in trainign df.

In [6]:
# https://stackoverflow.com/questions/4664850/how-to-find-all-occurrences-of-a-substring

# Code to find linebreaks in text.
# Sample string.
samp_string = "We need to identify 4 of these.\n\n\n\nAnd we need to find five later.\n\n\n\n\nBut not yet.\n\n\n\n\nNow."
chunks = [m.start() for m in re.finditer('\n\s*\n\s*\n\s*\n', samp_string)]   # Extract all occurences of string.
print(chunks)   # Pring chunks.
samp_string = samp_string[:chunks[-1]]   # Remove all after last.
samp_string = samp_string[chunks[0]:]   # Remove all before first.
print(samp_string)    # Print new text.

[31, 66, 83]




And we need to find five later.




But not yet.


In [7]:
f = open("/home/x-mbemus/Desktop/text_files/" + df.at[61, "file_name"])   # Open random file.
text = ' '.join(f.readlines())   # Read in file.
chunks = [m.start() for m in re.finditer('\n\s*\n\s*\n', text)]   # Find string occurences.
print(chunks)   # Print occurences.

[933, 996, 1086, 1419, 35212, 59430, 92793, 124165, 148347, 171760, 222362, 248457]


In [8]:
for i in df.index:   # For each text.
    # Track progress.
    clear_output(wait=True)
    print(f"Current Index: {i}/{len(df.index)-1}")
    try:
        # Open file.
        f = open("/home/x-mbemus/Desktop/text_files/" + df.at[i, "file_name"], "r", errors="ignore")
        text = ' '.join(f.readlines())   # Read in text.
    except:
        text = "\n \n \n \n \n \n \n"
    # Find triple linebreaks.
    chunks = [ind.start() for ind in re.finditer('\n\s*\n\s*\n', text)]
    text = text[:chunks[-1]]   # Remove everything after last triple-linebreak.
    text = text[chunks[0]:]   # Remove everything before first.
    text = re.sub(r"\\n|\\t", " ", text)   # Remove linebreaks and tabs..
    text = re.sub(r"\s+", " ", text)   # Remove extra spaces. 
    df.at[i, "full_text"] = text   # Save as full_text in df.

Current Index: 665/665


In [9]:
# Find min and max text lengths.
min_text = 999999999   # Start min.
max_text = 0   # Start max.
for i in df.index:   # For each text.
    t_len = len(df.at[i, "full_text"].split(" "))   # Read in tokens and count.
    if t_len > max_text:   # If longer than current max,
        max_text = t_len   # Assign as max.
        max_ind = i   # Save index.
    if t_len < min_text:   # If smaller than current min,
        min_text = t_len   # Assign as min.
        min_ind = i   # Save index.

# Print results. 
print(f"Longest Text: {max_text}\nLongest Document: {df.at[max_ind, 'title']}")
print(f"Smallest Text: {min_text}\nShortest Document: {df.at[min_ind, 'title']}")

Longest Text: 566193
Longest Document: War and Peace
Smallest Text: 1
Shortest Document: Alaska


In [10]:
# Repeat above process for summaries. 
smlst = 999999999
bigst = 0
for i in df.index:
    text = re.sub(r"\\n|\\t", " ", df.at[i, 'summary'])
    t_len = len(text.split(" "))
    if t_len > bigst:
        bigst = t_len
        big_ind = i
    if t_len < smlst:
        smlst = t_len
        sml_ind = i
print(f"Longest Summary: {bigst}\n\tDocument: {df.at[big_ind, 'title']}")
print(f"Smallest Summary: {smlst}\n\tDocument: {df.at[sml_ind, 'title']}")

Longest Summary: 5475
	Document: The Scapegoat
Smallest Summary: 4
	Document: The American Claimant


In [11]:
del df   # Remove training df.

In [12]:
df = pd.read_csv("/home/x-mbemus/Desktop/text_test_dataf.csv")   # Read in test dataframe.

In [13]:
# Show effect of removing flagged observations.
print(len(df.index))
print(len(df[df["flagged"] == 0].index))

# Remove flagged observations..
df = df[df["flagged"] == 0].reset_index()

167
100


In [14]:
for i in df.index:   # Read in text files.
    # Open file.
    f = open("/home/x-mbemus/Desktop/text_files/" + df.at[i, "file_name"], errors="ignore")
    text = ' '.join(f.readlines())   # Read in text.
    # Find triple-linebreaks.
    chunks = [ind.start() for ind in re.finditer('\n\s*\n\s*\n', text)]
    # Remove chunks as before.
    text = text[:chunks[-1]]
    text = text[chunks[0]:]
    # Preprocess linebreaks, tabs, and extra spaces.
    text = re.sub(r"\\n|\\t", " ", text)
    text = re.sub(r"\s+", " ", text)
    df.at[i, "full_text"] = text   # Save full text.

In [15]:
# Find longest and shortest texts as above.
min_text = 999999999
max_text = 0
for i in df.index:
    t_len = len(df.at[i, "full_text"].split(" "))
    if t_len > max_text and t_len:
        max_text = t_len
        max_ind = i
    if t_len < min_text:
        min_text = t_len
        min_ind = i
print(f"Longest Text: {max_text}\nLongest Document: {df.at[max_ind, 'title']}")
print(f"Smallest Text: {min_text}\nShortest Document: {df.at[min_ind, 'title']}")

Longest Text: 357844
Longest Document: Dombey and Son
Smallest Text: 57
Shortest Document: Palimpsest


In [16]:
# Find longest and shortest summaries, as above.
smlst_ts = 999999999
bigst_ts = 0
for i in df.index:
    text = re.sub(r"\\n|\\t", " ", df.at[i, 'summary'])
    t_len = len(text.split(" "))
    if t_len > bigst_ts:
        bigst_ts = t_len
        big_ind = i
    if t_len < smlst_ts:
        smlst_ts = t_len
        sml_ind = i
print(f"Longest Summary: {bigst_ts}\n\tDocument: {df.at[big_ind, 'title']}")
print(f"Smallest Summary: {smlst_ts}\n\tDocument: {df.at[sml_ind, 'title']}")

Longest Summary: 5475
	Document: The Scapegoat
Smallest Summary: 43
	Document: Joseph and His Friend: A Story of Pennsylvania


LED

In [17]:
# Code borrowed from https://huggingface.co/pszemraj/led-base-book-summary

import torch
from transformers import pipeline

hf_name = "pszemraj/led-base-book-summary"

summarizer = pipeline(
    "summarization",
    hf_name,
    device=0 if torch.cuda.is_available() else -1,
)

In [18]:
# More code from https://huggingface.co/pszemraj/led-base-book-summary

wall_of_text = "your words here"

result = summarizer(
    wall_of_text,
    min_length=8,
    max_length=256,
    no_repeat_ngram_size=3,
    encoder_no_repeat_ngram_size=3,
    repetition_penalty=3.5,
    num_beams=4,
    do_sample=False,
    early_stopping=True,
)
print(result[0]['summary_text'])

Your max_length is set to 256, but your input_length is only 5. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


You've said a lot of stuff here.


In [19]:
#print(df.at[0, "full_text"])

In [25]:
# Dict to hold results. 
results = {"title":[], "target":[], "pred":[], "bleu":[], "rouge":[]}
diver = len(df.at[10, "full_text"].split(" "))/8000   # Sample diver.
max_inp = 9500   # Max input size.

for j in range(0, 60):   # For each in first 60.
    # Check if text is shorter than max input.
    if len(df.at[j, "full_text"].split(" ")) < max_inp:
        # If within range, summarize.
        test_res = summarizer(
            df.at[j, "full_text"],
            min_length=smlst,
            max_length=bigst,
            no_repeat_ngram_size=3,
            encoder_no_repeat_ngram_size=3,
            repetition_penalty=3.5,
            num_beams=4,
            do_sample=False,
            early_stopping=True,
        )
        # Extract summary.
        sub_sum = test_res[0]["summary_text"]
        # Update progress.
        clear_output(wait=True)
        print(f"Current Summary: {i+1}/{len(df.index)}")
        
    else:   # If outside of range.
        sub_sum = ""   # Empty string for summary.
        diver = len(df.at[j, "full_text"].split(" "))/max_inp   # Length to divide text by.
        max_div = diver//1 + 1   # Number of divisions.
    
        for i in range(0, int(max_div)):   # For each dividsion.
            if i != max_div-1:    # If not last division.
                # Select text within range.
                new_text = " ".join(df.at[j, "full_text"].split(" ")[i*max_inp:(i+1)*max_inp])
            else:   # If last division.
                # Select all remaining text.
                new_text = " ".join(df.at[j, "full_text"].split(" ")[max_inp*(i+1):])
            
            # Summarize snew_text.
            test_res = summarizer(
                new_text,
                min_length=round(smlst/diver),
                max_length=round(bigst/diver),
                no_repeat_ngram_size=3,
                encoder_no_repeat_ngram_size=3,
                repetition_penalty=3.5,
                num_beams=4,
                do_sample=False,
                early_stopping=True,
            )
            # Add sub-summary to full summary.
            sub_sum += test_res[0]['summary_text'] + " "
            
            # Track progress.
            clear_output(wait=True)
            print(f"Current Summary: {j+1}/{len(df.index)}")
            print(f"{i+1}/{max_div} Iterations Complete")

    # Extract target summary.
    target = df.at[j, "summary"]
    full_output = sub_sum.strip()   # Strip extra spaces.
    
    # Compute metrics.
    blue = bleu_scorer.sentence_score(hypothesis=full_output, references=[target])
    red = rouge_scorer.get_scores(hyps=full_output, refs=target)
    
    # Add features to dict.
    results["title"].append(df.at[j, "title"])
    results["target"].append(target)
    results["pred"].append(full_output)
    results['bleu'].append(blue.score)
    results['rouge'].append(red[0]["rouge-l"]["f"])
        
# Save results to data frame.
test_df = pd.DataFrame(results)
# Save df as csv.
test_df.to_csv("/home/x-mbemus/Desktop/led_test_result3a.csv", index=False)

Current Summary: 60/100
22/22.0 Iterations Complete


In [31]:
# Attempt to repeat above process for 60:100.
# Consistently ran into errors. Could only summarize a few here. 
# Long run time.
results = {"title":[], "target":[], "pred":[], "bleu":[], "rouge":[]}
diver = len(df.at[10, "full_text"].split(" "))/8000
max_inp = 9500

for j in list(range(60, 84)) + list(range(87, len(df.index))):   # Experiencing errors in this range.
    try:
        if len(df.at[j, "full_text"].split(" ")) < max_inp:
            test_res = summarizer(
                df.at[j, "full_text"],
                min_length=smlst,
                max_length=bigst,
                no_repeat_ngram_size=3,
                encoder_no_repeat_ngram_size=3,
                repetition_penalty=3.5,
                num_beams=4,
                do_sample=False,
                early_stopping=True,
            )
            sub_sum = test_res[0]["summary_text"]
            clear_output(wait=True)
            print(f"Current Summary: {j+1}/{len(df.index)}")
        
        else:
            sub_sum = ""
            diver = len(df.at[j, "full_text"].split(" "))/max_inp
            max_div = diver//1 + 1
    
            for i in range(0, int(max_div)):
                if i != max_div-1: 
                    new_text = " ".join(df.at[j, "full_text"].split(" ")[i*max_inp:(i+1)*max_inp])
                else:
                    new_text = " ".join(df.at[j, "full_text"].split(" ")[max_inp*(i+1):])
        
                test_res = summarizer(
                    new_text,
                    min_length=round(smlst/diver),
                    max_length=round(bigst/diver),
                    no_repeat_ngram_size=3,
                    encoder_no_repeat_ngram_size=3,
                    repetition_penalty=3.5,
                    num_beams=4,
                    do_sample=False,
                    early_stopping=True,
                )
        target = df.at[j, "summary"]
        full_output = sub_sum.strip()
        blue = bleu_scorer.sentence_score(hypothesis=full_output, references=[target])
        red = rouge_scorer.get_scores(hyps=full_output, refs=target)
        results['bleu'].append(blue.score)
        results['rouge'].append(red[0]["rouge-l"]["f"])
        results["title"].append(df.at[j, "title"])
        results["target"].append(target)
        results["pred"].append(full_output)
        
    except:
        results['bleu'].append(0)
        results['rouge'].append(0)
        results["title"].append(df.at[j,"title"])
        results["target"].append(df.at[j,"summary"])
        results['pred'].append("")
    
        
test_df = pd.DataFrame(results)
test_df.to_csv("/home/x-mbemus/Desktop/led_test_result3b.csv", index=False)

Current Summary: 74/100


Your max_length is set to 2578, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 1464, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 326, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 1470, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_

In [17]:
# 10 Summaries takes about 11 min 53 sec. Most summaries < 1:10. All below 1:20.
len(df.index)
# Should take about 3 and a half hours.

167

FalconsAI

In [14]:
# Sample code from https://huggingface.co/Falconsai/text_summarization

from transformers import pipeline

summarizer = pipeline("summarization", model="Falconsai/text_summarization")

ARTICLE = """ 
Hugging Face: Revolutionizing Natural Language Processing
Introduction
In the rapidly evolving field of Natural Language Processing (NLP), Hugging Face has emerged as a prominent and innovative force. This article will explore the story and significance of Hugging Face, a company that has made remarkable contributions to NLP and AI as a whole. From its inception to its role in democratizing AI, Hugging Face has left an indelible mark on the industry.
The Birth of Hugging Face
Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf. The name "Hugging Face" was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug. Initially, they began as a chatbot company but later shifted their focus to NLP, driven by their belief in the transformative potential of this technology.
Transformative Innovations
Hugging Face is best known for its open-source contributions, particularly the "Transformers" library. This library has become the de facto standard for NLP and enables researchers, developers, and organizations to easily access and utilize state-of-the-art pre-trained language models, such as BERT, GPT-3, and more. These models have countless applications, from chatbots and virtual assistants to language translation and sentiment analysis.
Key Contributions:
1. **Transformers Library:** The Transformers library provides a unified interface for more than 50 pre-trained models, simplifying the development of NLP applications. It allows users to fine-tune these models for specific tasks, making it accessible to a wider audience.
2. **Model Hub:** Hugging Face's Model Hub is a treasure trove of pre-trained models, making it simple for anyone to access, experiment with, and fine-tune models. Researchers and developers around the world can collaborate and share their models through this platform.
3. **Hugging Face Transformers Community:** Hugging Face has fostered a vibrant online community where developers, researchers, and AI enthusiasts can share their knowledge, code, and insights. This collaborative spirit has accelerated the growth of NLP.
Democratizing AI
Hugging Face's most significant impact has been the democratization of AI and NLP. Their commitment to open-source development has made powerful AI models accessible to individuals, startups, and established organizations. This approach contrasts with the traditional proprietary AI model market, which often limits access to those with substantial resources.
By providing open-source models and tools, Hugging Face has empowered a diverse array of users to innovate and create their own NLP applications. This shift has fostered inclusivity, allowing a broader range of voices to contribute to AI research and development.
Industry Adoption
The success and impact of Hugging Face are evident in its widespread adoption. Numerous companies and institutions, from startups to tech giants, leverage Hugging Face's technology for their AI applications. This includes industries as varied as healthcare, finance, and entertainment, showcasing the versatility of NLP and Hugging Face's contributions.
Future Directions
Hugging Face's journey is far from over. As of my last knowledge update in September 2021, the company was actively pursuing research into ethical AI, bias reduction in models, and more. Given their track record of innovation and commitment to the AI community, it is likely that they will continue to lead in ethical AI development and promote responsible use of NLP technologies.
Conclusion
Hugging Face's story is one of transformation, collaboration, and empowerment. Their open-source contributions have reshaped the NLP landscape and democratized access to AI. As they continue to push the boundaries of AI research, we can expect Hugging Face to remain at the forefront of innovation, contributing to a more inclusive and ethical AI future. Their journey reminds us that the power of open-source collaboration can lead to groundbreaking advancements in technology and bring AI within the reach of many.
"""
print(summarizer(ARTICLE, max_length=230, min_length=30, do_sample=False)[0]["summary_text"])

Token indices sequence length is longer than the specified maximum sequence length for this model (851 > 512). Running this sequence through the model will result in indexing errors


Hugging Face has emerged as a prominent and innovative force in NLP . From its inception to its role in democratizing AI, the company has left an indelible mark on the industry . The name "Hugging Face" was chosen to reflect the company's mission of making AI models more accessible and friendly to humans .


In [16]:
# Adapt code from above.
max_inp = 512   # Maximum input length.
# Dict to hold results.
results = {"title":[], "target":[], "pred":[], "bleu":[], "rouge":[]}

for j in df.index:   # Iterate through df.
    if len(df.at[j, "full_text"].split(" ")) > max_inp:   # If text longer than max_inp,
        
        sub_sum = ""   # Empty string to hold total summary.
        diver = len(df.at[j, "full_text"].split(" "))/max_inp   # Find dividsion length.
        max_div = diver//1 + 1   # Count number of divisions.
    
        for i in range(0, int(max_div)):   # For each subdivision
            if i != max_div-1:    # If not last division.
                # Pick certain range out of text.
                new_text = " ".join(df.at[j, "full_text"].split(" ")[i*max_inp:(i+1)*max_inp])
            else:   # If last divison.
                # Take remaining. 
                new_text = " ".join(df.at[j, "full_text"].split(" ")[max_inp*(i+1):])
        
            # Summarize sub-part of text.
            test_res = summarizer(new_text, max_length=round(bigst/diver), min_length=round(smlst/diver), do_sample=False)
            sub_sum += test_res[0]['summary_text'] + " "   # Add to total summary.
            
            # Track progress.
            clear_output(wait=True)
            print(f"Current Summary: {j+1}/{len(df.index)}")
            print(f"{i+1}/{max_div} Iterations Complete")
    
    else:   # If text length fits in summarizer.
        # Summarize text.
        test_res = summarizer(df.at[j, "full_text"], max_length=bigst, min_length=smlst, do_sample=False)
        sub_sum = test_res[0]['summary_text']

    target = df.at[j, "summary"]   # Extract target response.
    full_output = sub_sum.strip()   # Remove extra end spaces.
    
    # Get scores.
    blue = bleu_scorer.sentence_score(hypothesis=full_output, references=[target])
    red = rouge_scorer.get_scores(hyps=full_output, refs=target)
    
    # Add to dict.
    results["title"].append(df.at[j, "title"])
    results["target"].append(target)
    results["pred"].append(full_output)
    results['bleu'].append(blue.score)
    results['rouge'].append(red[0]["rouge-l"]["f"])

# Convert dict to df.
test_df = pd.DataFrame(results)
# Save df. 
test_df.to_csv("/home/x-mbemus/Desktop/falcon_test_result2.csv", index=False)

Current Summary: 100/100
163/163.0 Iterations Complete


In [ ]:
# Takes about 2 min per summary, 167 summaries, 334 minutes or 5 and 1/2 hours ish.

Pegasus

In [19]:
#!pip install sentencepiece
import sentencepiece

In [20]:
# Sample code from https://towardsdatascience.com/how-to-perform-abstractive-summarization-with-pegasus-3dd74e48bafb

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
src_text = [
    """ PG&E stated it scheduled the blacokouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the damage incurred by the potentially hazardous conditions, which could cause fires if an outage were to occur naturally. """
]

model_name = "google/pegasus-xsum"
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
batch = tokenizer.prepare_seq2seq_batch(src_text, truncation=True, padding="longest", return_tensors="pt").to(torch_device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text[0])

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/x-mbemus/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3982: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your sp

California's largest electricity provider has announced it will cut power to more than 300,000 homes and businesses over the next few days.


In [21]:
# Dict to hold results.
results = {"title":[], "target":[], "pred":[], "bleu":[], "rouge":[]}

for i in df.index:   # For each test observation.
    # Tokenize text.
    batch = tokenizer.prepare_seq2seq_batch(df.at[i, "full_text"], 
                                            truncation=True, 
                                            padding="longest", 
                                            return_tensors="pt").to(torch_device)
    translated = model.generate(**batch)   # Generate summary.
    mod_out = tokenizer.batch_decode(translated, skip_special_tokens=True)   # Decode summary.
    
    target = df.at[i, "summary"]   # Extract target.
    full_output = mod_out[0]   # Extract output.
    
    # Compute scores.
    blue = bleu_scorer.sentence_score(hypothesis=full_output, references=[target])
    red = rouge_scorer.get_scores(hyps=full_output, refs=target)
    
    # Save results.
    results["title"].append(df.at[i, "title"])
    results["target"].append(target)
    results["pred"].append(full_output)
    results['bleu'].append(blue.score)
    results['rouge'].append(red[0]["rouge-l"]["f"])
    
    # Track progress. 
    clear_output(wait=True)
    print(f"{i+1}/{len(df.index)} Summaries Complete")

# Convert dict to df.
result_df = pd.DataFrame(results)
# Save df. 
result_df.to_csv("/home/x-mbemus/Desktop/pegasus_test_result2.csv", index=False)

100/100 Summaries Complete


In [43]:
# Open test file.
pegasus_test = pd.read_csv("/home/x-mbemus/Desktop/pegasus_test_result.csv")
pegasus_test.head()

,title,target,pred,bleu,rouge
0,Evolution,The book follows the evolution of mankind as ...,In our series of letters from British journali...,2.631998e-04,0.048309
1,Cause of Death,New Year's Eve and the final murder scene of ...,In our series of letters from British journali...,2.112223e-01,0.057692
2,Cursed,Various demons have battled Spike since he wa...,In our series of letters from British journali...,3.928301e-01,0.113636
3,Death in Venice,"The main character is Gustav von Aschenbach, ...",In our series of letters from British journali...,4.048940e-17,0.020906
4,The Message,Cassie and Tobias are having strange dreams a...,In our series of letters from British journali...,9.757962e-08,0.037453


In [45]:
pegasus_test['bleu'].mean()   # Compute mean bleu.

0.063126158880524

In [46]:
pegasus_test['rouge'].mean()   # Compute mean rouge.

0.04204537295642239

In [47]:
# Open falcon test results.
falcon_test = pd.read_csv("/home/x-mbemus/Desktop/falcon_test_result2.csv")
falcon_test.head()

,title,target,pred,bleu,rouge
0,Evolution,The book follows the evolution of mankind as ...,This eBook is for the use of anyone anywhere i...,0.834725,0.128527
1,Cause of Death,New Year's Eve and the final murder scene of ...,Title: Cause of Death Author: Max Tadlock Rele...,0.314718,0.057143
2,Cursed,Various demons have battled Spike since he wa...,This eBook is for the use of anyone anywhere i...,0.111076,0.043668
3,Death in Venice,"The main character is Gustav von Aschenbach, ...",Gustav Aschenbach had left his apartment on th...,1.018421,0.171306
4,The Message,Cassie and Tobias are having strange dreams a...,Honore de Balzac Translated by Ellen Marriage ...,0.547090,0.123119


In [48]:
falcon_test['bleu'].mean()   # Compute mean BLEU.

1.1117372780978352

In [49]:
falcon_test['rouge'].mean()   # Compute mean ROUGE. 

0.1242950205464541